In [2]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time

In [3]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [4]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [5]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(size,784)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(size,784)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

In [6]:
from nnet.network import Sequential,layers
from nnet.layers import conv2d,max_pool,flatten,dense,dropout,Activation
from nnet import functions
from nnet import optimizers
import numpy as np

Seed: 151
Seed: 408


In [7]:
model=Sequential()
model.add(dense(128,input_shape=(784),activation=functions.sigmoid))
model.add(dense(32,activation=functions.sigmoid))
model.add(dense(10,activation=functions.sigmoid))
# model.add(Activation(functions.sigmoid))

In [8]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
0 input_layer(InputLayer) (None, 784)                echo             0
__________________________________________________________________________________________
1 dense(dense)            (None, 128)                sigmoid          100480
__________________________________________________________________________________________
2 dense(dense)            (None, 32)                 sigmoid          4128
__________________________________________________________________________________________
3 dense(dense)            (None, 10)                 sigmoid          330
Total Params: 104,938
Trainable Params: 104,938
Non-trainable Params: 0


In [9]:
model.compile(optimizer=optimizers.iterative,loss=functions.mean_squared_error,learning_rate=0.1)

In [10]:
btsz=2
inp,y_inp=batch_gen(0,size=btsz)
logits=model.fit(inp,y_inp)

In [11]:
X_inp=inp
labels=y_inp
for obj in model.sequence:
	X_inp=obj.forward(X_inp)
err=model.del_loss(X_inp,labels)

In [12]:
model.predict(inp)-y_inp

array([[ 0.5020977 ,  0.4387907 ,  0.4360888 ,  0.44107332,  0.45350362,
        -0.50795789,  0.45003523,  0.43680118,  0.45020728,  0.44937507],
       [-0.49790969,  0.43879869,  0.43610379,  0.4410947 ,  0.4534963 ,
         0.49204316,  0.45006011,  0.43680938,  0.45020762,  0.44938554]])

In [13]:
i=model.lenseq_m1
for obj in model.sequence[::-1]:
	err=obj.backprop(err,layer=i)
	i-=1
model.optimizer(model.sequence,model.learning_rate,model.beta)

In [14]:
logits.argmax(axis=1),y_inp.argmax(axis=1)

(array([4, 4]), array([5, 0]))

In [19]:
logits.sum(axis=0,keedims=True).shape

TypeError: _sum() got an unexpected keyword argument 'keedims'

In [45]:
def validate():
    vdsz=2000
    dvme=10000//vdsz
    acc=0
    cross_entropy_loss=0
    for smpl in range(0,10000,vdsz):
        print("\rCalculating Validation acc...",smpl//vdsz+1,end='')
        inp,y_inp=batch_gen(smpl,size=vdsz,validate=True)
        logits=model.predict(inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc+=100*(ans==cor).mean()
        cross_entropy_loss+=model.loss(logits,labels=y_inp).mean()*10
    print("\rValidation Acc: {:.3f} %        Val loss: {:.8f}".format(acc/dvme,cross_entropy_loss/dvme))
    logits=model.predict(inp[:1])

In [46]:
def run():
    st=0
    btsz=16
    acc_tn=0
    _cc=0
    loss_tn=0
    while st<=60000:
        sam_tm=time()
        perc=st/600
        ck=np.random.randint(0,60000-btsz)
        inp,y_inp=batch_gen(ck,size=btsz)
        logits=model.fit(inp,y_inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc=100*(ans==cor).mean()
        cross_entropy_loss=model.loss(logits=logits,labels=y_inp).mean()*10
        acc_tn+=acc
        _cc+=1
        loss_tn+=cross_entropy_loss
        acc=acc_tn/_cc
        loss_=loss_tn/_cc
        if acc>=98:
            model.learning_rate=1e-5
        elif acc>=97:
            model.learning_rate=1e-4
#         elif acc>=95:
#             model.learning_rate=1e-4
        sam_tm=time()-sam_tm
        rem_sam=(60000-st)/btsz
        eta=int(rem_sam*sam_tm)
        print("\rProgress: {:.2f} %    Acc: {:.3f} %    loss: {:.6f}     Sample time: {:.3f}s    ETA: {}:{}s    .".format(perc,acc,loss_,sam_tm,eta//60,eta%60),end='')
        st+=btsz
    print("")

In [47]:
epochs=3
for epoch in range(epochs):
    print("EPOCH:",epoch+1,'/',epochs)
    st_tm=time()
    run()
    print("Epoch time: {}:{}s".format(int(time()-st_tm)//60,int(time()-st_tm)%60))
    validate()

EPOCH: 1 / 3


Progress: 4.96 %    Acc: 10.160 %    loss: 0.456380     Sample time: 0.003s    ETA: 0:9s    .

Progress: 100.00 %    Acc: 21.029 %    loss: 0.412667     Sample time: 0.001s    ETA: 0:0s    .
Epoch time: 0:8s
Validation Acc: 73.020 %        Val loss: 0.20810102
EPOCH: 2 / 3
Progress: 100.00 %    Acc: 88.621 %    loss: 0.091433     Sample time: 0.003s    ETA: 0:0s    .
Epoch time: 0:12s
Validation Acc: 93.020 %        Val loss: 0.05475213
EPOCH: 3 / 3
Progress: 100.00 %    Acc: 94.440 %    loss: 0.045392     Sample time: 0.001s    ETA: 0:0s    .
Epoch time: 0:11s
Validation Acc: 95.170 %        Val loss: 0.03959225
